In [1]:
import requests
import pystac
import datetime
import json

In [25]:
# initialize a new STAC cat

catalog = pystac.Catalog('3dep', 
                         'A catalog of USGS 3DEP Lidar hosted on AWS s3.',
                         href='https://3dep-stac.s3.amazonaws.com/catalog.json',
                         stac_extensions=['POINTCLOUD'])

In [5]:
# fetch the boundaries as geojson

boundaries = requests.get('https://raw.githubusercontent.com/hobu/usgs-lidar/master/boundaries/resources.geojson').json()

In [8]:
from shapely.geometry import shape, mapping
from pathlib import Path
from tqdm import tqdm 

base = Path('./STAC/3dep')
base.mkdirs(exist_ok=True, parents=True)

data = []

for f in tqdm(boundaries['features'], ncols=100):
    props = f['properties']
    geom = shape(f['geometry'])
    ept = requests.get(props['url']).json()
    data.append((props, geom, ept))

In [28]:
# loop over the features and create a stac item for each...

for f in tqdm(data, ncols=100):
    props, geom, ept = f
    
    item = pystac.Item(props['name'], 
                       mapping(geom), 
                       list(geom.bounds), 
                       datetime.datetime.now(), 
                       {'description': 'A USGS Lidar pointcloud in entwine format'})
    
    item.ext.enable(pystac.Extensions.POINTCLOUD)
    item.ext.pointcloud.apply(props['count'], 'lidar', 'entwine', ept['schema'], epsg='EPSG:3857')
    
    asset = pystac.Asset(props['url'], 'entwine', 'The ept.json for accessing data')
    item.add_asset('ept.json', asset)

    item_link = pystac.Link('self', f'https://3dep-stac.s3.amazonaws.com/{props["name"]}.json')
    item_parent = pystac.Link('parent', f'https://3dep-stac.s3.amazonaws.com/catalog.json')
    item.add_links([item_link, item_parent])

    with open(base / f"{props['name']}.json", 'w') as f:
        d = item.to_dict()
        json.dump(d, f)

    link = pystac.Link('item', f'https://3dep-stac.s3.amazonaws.com/{props["name"]}.json')
    catalog.add_link(link)

with open(base / "catalog.json", 'w') as f:
    json.dump(catalog.to_dict(), f)

100%|██████████████████████████████████████████████████████████| 1270/1270 [00:01<00:00, 941.36it/s]
